In [ ]:
import os
import glob
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import AMD_Tools3 as amd

In [ ]:
def meshcode2lat(meshcode):
    latlon = amd.mesh2lalo(meshcode)
    return latlon[0]

def meshcode2lon(meshcode):
    latlon = amd.mesh2lalo(meshcode)
    return latlon[1]

In [ ]:
#load simdata

#sh (header)
head_lines = []
for prefyear in tqdm(glob.glob('simdata/*/*'), desc='header'):
    head_line = subprocess.check_output(f'head -n 1 {prefyear}/*.csv', shell=True)
    head_line = head_line.decode('utf-8')
    head_line = head_line.split('\n')
    head_lines += head_line
# print(head_lines[:10])

#extract header
header = head_lines[1].split(',')
for l in range(0, len(head_lines), 3):
    assert(header == head_lines[l+1].split(','))  # assert all simdata has the same header

#sh (values)
last_lines = []
for prefyear in tqdm(glob.glob('simdata/*/*'), desc='value'):
    last_line = subprocess.check_output(f'tail -n 1 {prefyear}/*.csv', shell=True)
    last_line = last_line.decode('utf-8')
    last_line = last_line.split('\n')
    last_lines += last_line
# print(last_lines[:10])

#dvi
simdata = []
for l in trange(0, len(last_lines), 3, desc='combine'):
    filename = last_lines[l]
    values = last_lines[l+1]
    assert('==>' in filename)
    
    filename2 = filename.split(' ')[1]
    _, pref, year, meshcode = os.path.splitext(filename2)[0].split('/')
    values = values.split(',')
    # print(pref, year, meshcode)
    simdatum = dict(zip(header, values))
    simdatum['pref'] = pref
    simdatum['meshcode'] = meshcode
    simdatum['year'] = int(year)
    simdata.append(simdatum)

#to dataframe
simdata = pd.DataFrame(simdata)
for dt in ('DVI', 'GY', 'LAI', 'TMX', 'TAV', 'RAD'):
    simdata[dt] = pd.to_numeric(simdata[dt])
print('simdata.shape:', simdata.shape)
simdata.head()

# 1km-mesh sampling interval

In [ ]:
#diff of longtitude
for pref in simdata.pref.unique():
    print('pref:', pref)
    # lats = [meshcode2lat(meshcode) for meshcode in simdata[simdata.pref==pref].meshcode]
    lons = [meshcode2lon(meshcode) for meshcode in simdata[simdata.pref==pref].meshcode]
    print(np.diff(lons[:5]))

In [ ]:
#scatter plot
fig, ax = plt.subplots(figsize=(7, 7))
year = np.random.randint(1980, 2016)
df = simdata[simdata.year == year].copy()
df['lat'] = df.meshcode.astype(str).apply(meshcode2lat)
df['lon'] = df.meshcode.astype(str).apply(meshcode2lon)
df.plot.scatter(x='lon', y='lat', ax=ax)
ax.set_title(f'year={year}')
fig.tight_layout()

# DVI variation


In [ ]:
simdata.DVI.round(1).hist(bins=20).set(xlabel='DVI', ylabel='Frequency')

In [ ]:
simdata.groupby(['pref']).DVI.describe()

In [ ]:
simdata.groupby(['year']).DVI.describe()

In [ ]:
simdata.groupby(['pref', 'year']).DVI.describe()

# GY Heatmap

In [ ]:
param = 'GY'

In [ ]:
simdata.groupby(['pref'])[param].describe()

In [ ]:
#groupby
simdata_grouped = simdata.groupby(by=['meshcode', 'year'])[param].max()
# print('len(DVI<1.9):', (simdata_grouped < 1.9).sum())
# print('len(simdata):', len(simdata_grouped))
simdata2 = simdata_grouped.reset_index()
simdata2['lat'] = simdata2.meshcode.astype(str).apply(meshcode2lat)
simdata2['lon'] = simdata2.meshcode.astype(str).apply(meshcode2lon)
simdata2.head()

In [ ]:
#scatter plot
from matplotlib import cm
import seaborn as sns
fig, axes = plt.subplots(9, 4, figsize=(10, 15))
for year, ax in tqdm(zip(simdata2.year.unique(), axes.flatten())):
    df = simdata2[simdata2.year == year].copy()
    piv = pd.pivot_table(df, values='GY', index='lat', columns='lon')
    sns.heatmap(piv, ax=ax, cmap=cm.hot,
                vmin=0, vmax=simdata[param].max(),
                # vmin=0, vmax=800,
                xticklabels=[], yticklabels=[])
    ax.set_title(f'year={year}', fontsize=9)
    ax.set_xlabel('')
    ax.set_ylabel('')
    # ax.get_xaxis().set_visible(False)
    # ax.get_yaxis().set_visible(False)
    ax.invert_yaxis()
    # break
fig.tight_layout()
fig.savefig('output/vis_GYmap.png')

In [ ]:
#on google map
import folium
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def value_to_hex(value, vmin, vmax):
    cmap = cm.hot
    norm = Normalize(vmin=vmin, vmax=vmax)
    rgb = cmap(norm(value))[:3]
    r, g, b = (np.array(rgb) * 255).astype(int)
    hex = "#{0:02x}{1:02x}{2:02x}".format(r, g, b)
    return hex

year = np.random.randint(1980, 2015)
print('year:', year)

map = folium.Map(location=[35.1708333, 138.18125], zoom_start=6)
df = simdata2[simdata2.year==year].copy()

for i, row in tqdm(df.iterrows()):
    
    if i % 10 == 0:
        color = value_to_hex(row[param], vmin=0, vmax=900)
        folium.CircleMarker([row['lat'], row['lon']], radius=1,
                    popup=str(row[param]), color=color).add_to(map)
map

# Whey DVI finished in 1.9 in some cases?

In [ ]:
print('Just because environmental data is given until 10/31')
simdata[(simdata.DVI > 1.9) & (simdata.DVI < 2.0)][['DVI', 'date']]

# Input data

In [ ]:
#load all simdata (take time...)
simdata_all = [pd.read_csv(csvpath) for csvpath in glob.glob('simdata/*/*/*.csv')]
simdata_all = pd.concat(simdata_all, axis=0)

print('simdata_all.shape:', simdata_all.shape)
simdata_all.head()

In [ ]:
#Histogram of inputs

dt = ['DVI', 'DVR', 'TAV', 'TMX', 'RAD', 'DL', 'PPM']
fig, axes = plt.subplots(2, 4, figsize=(12, 5))
cnt = 0

for _dt in tqdm(dt):
    ax = axes.flatten()[cnt]
    df = simdata_all[_dt]
    df.plot.hist(ax=ax)
    ax.set_xlabel(_dt)
    ax.set_ylabel('Frequency')
    cnt += 1
    
fig.tight_layout()

In [ ]:
#Scatter matrix of inputs

import seaborn as sns
sns.set(style="ticks")

dt = ['DVI', 'DVR', 'TAV', 'TMX', 'RAD', 'DL', 'PPM']
df = simdata_all[dt]
df = df.sample(frac=.005, axis=0)  # random sampling to make it fast
print('df.shape:', df.shape)
sns.pairplot(df, size=1.)